In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
print(os.environ.get('OPENAI_MODEL_NAME'))

gpt-4o-mini


In [4]:
from crewai import Agent, LLM

schema_creator_agent = Agent(
    role='Schema Creator',
    goal='Generate a SQLite database schema that includes relevant fields and data types based on a given business use case.',
    backstory="""You are an expert database architect with extensive experience in designing database schemas.
Your mission is to analyze business requirements and translate them into efficient SQLite table schemas.
You focus on ensuring data integrity, normalization, and optimal storage of data related to the business use case.""",
    llm=LLM(model="ollama/llama3.1:70b", base_url="http://localhost:11434"),  # Optional: Defaults to the model specified in OPENAI_MODEL_NAME environment variable
    tools=[],  # Optional: Add any tools the agent might need
    max_iter=25,  # Optional: Maximum iterations the agent can perform
    verbose=False,  # Optional: Set to True to enable detailed logs
    allow_delegation=False,  # Optional: Whether the agent can delegate tasks
    # Additional optional parameters can be added as needed
)

In [5]:
from crewai import Task

# Assume 'schema_creator_agent' is already defined as per previous code
# Define the business use case
business_use_case = "An e-commerce platform needs to store customer order information, including customer details, order items, quantities, prices, and order dates."

# Create the Schema Generation Task
schema_generation_task = Task(
    description=(
        f"Based on the following business use case, generate a SQLite database schema:\n\n"
        f"{business_use_case}"
    ),
    agent=schema_creator_agent,
    expected_output=(
        "A SQLite CREATE TABLE statement defining a table with relevant fields and data types "
        "to store data related to the given business use case."
    ),
    tools=[],  # Optional: Add any tools the agent might need
    async_execution=False,  # Optional: Execute synchronously
    context=[],  # Optional: No prior tasks needed for context
    human_input=False,  # Optional: No human review required
    # Additional optional parameters can be added as needed
)

In [6]:
from crewai import Crew

# Instantiate the crew with the agent and task
crew = Crew(
    agents=[schema_creator_agent],
    tasks=[schema_generation_task],
    verbose=True  # Optional: Set to True to enable detailed logs
)

# Kick off the crew to execute the task
crew_result = crew.kickoff()

# Access the output of the task
schema_output = schema_generation_task.output.raw
print("Generated SQLite Schema:")
print(schema_output)

# Agent: Schema Creator
## Task: Based on the following business use case, generate a SQLite database schema:

An e-commerce platform needs to store customer order information, including customer details, order items, quantities, prices, and order dates.


# Agent: Schema Creator
## Final Answer: 
```sql
CREATE TABLE Customers (
  CustomerID INTEGER PRIMARY KEY,
  FirstName TEXT NOT NULL,
  LastName TEXT NOT NULL,
  Email TEXT UNIQUE NOT NULL,
  Address TEXT,
  City TEXT,
  State TEXT,
  PostalCode TEXT,
  Country TEXT
);

CREATE TABLE Orders (
  OrderID INTEGER PRIMARY KEY,
  CustomerID INTEGER NOT NULL,
  OrderDate DATE NOT NULL,
  TotalCost REAL NOT NULL,
  FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

CREATE TABLE Products (
  ProductID INTEGER PRIMARY KEY,
  ProductName TEXT NOT NULL,
  Price REAL NOT NULL
);

CREATE TABLE OrderItems (
  OrderItemID INTEGER PRIMARY KEY,
  OrderID INTEGER NOT NULL,
  ProductID INTEGER NOT NULL,
  Quantity INTEGER NOT NULL,
  UnitPri

#### Link to ChatGPT thread

https://chatgpt.com/share/6711db06-23c0-8001-a416-a7b675f4e89f